In [1]:
library(ggplot2)
library(dplyr)
library(ISLR2)
options(warn=-1)
library(dvmisc)
library(splines)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: rbenchmark



In [2]:
bikedata = read.csv("Bike-Sharing-Dataset//hour.csv", header = TRUE)
bikedata$dteday = as.Date(bikedata$dteday, format = "%Y-%m-%d") # converting date column to appropriate data type
attach(bikedata)
head(bikedata)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
,<int>,<date>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
1,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
2,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
3,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
4,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
5,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
6,6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1


# Create new data frame with rows between hours 1 and 4 for Dec, Jan, Feb to show non-constant variance across data

In [16]:
temp1 <- bikedata %>%
  filter(hr >= 1 & hr <= 4 & mnth %in% c(12, 1, 2))

# Calculate mean and variance of cnt variable for new data frame
mean_cnt <- mean(temp1$cnt)
var_cnt <- var(temp1$cnt)

# Print the results
cat("Mean (1am - 4am in Dec, Jan, Feb):", mean_cnt, "\n")
cat("Variance (1am - 4am in Dec, Jan, Feb):", var_cnt, "\n")

Mean (1am - 4am in Dec, Jan, Feb): 12.10479 
Variance (1am - 4am in Dec, Jan, Feb): 284.7236 


# Create new data frame with rows between hours 6-9 for Apr, May, June to show non-constant variance to show non-constant variance across data

In [8]:
temp2 <- bikedata %>%
  filter(hr >= 6 & hr <= 9 & mnth %in% c(4, 5, 6))

# Calculate mean and variance of cnt variable for new data frame
mean_cnt <- mean(temp2$cnt)
var_cnt <- var(temp2$cnt)

# Print the results
cat("Mean of cnt for hours 6-9 in Apr, May, Jun:", mean_cnt, "\n")
cat("Variance of cnt for hours 6-9 in Apr, May, Jun:", var_cnt, "\n")


Mean of cnt for hours 6-9 in Apr, May, Jun: 238.0165 
Variance of cnt for hours 6-9 in Apr, May, Jun: 32062.56 


In [12]:
year_2011 <- subset(bikedata, yr == 0)
cnt_summary_2011 <- aggregate(cnt ~ mnth, data = year_2011, FUN = function(x) c(mean = mean(x), var = var(x)))
cnt_summary_2011$mean <- cnt_summary_2011$cnt[,1]
cnt_summary_2011$var <- cnt_summary_2011$cnt[,2]
cnt_summary_2011$yr <- 2011
cnt_summary_2011$yr <- factor(cnt_summary_2011$yr) 
cnt_summary_2011 = cnt_summary_2011[c("mnth", "mean", "var", "yr")]
cnt_summary_2011

mnth,mean,var,yr
<int>,<dbl>,<dbl>,<fct>
1,55.50727,2363.968,2011
2,74.29122,4048.268,2011
3,87.73288,6221.974,2011
4,131.94715,15056.421,2011
5,182.55511,21431.031,2011
6,199.32222,24050.936,2011
7,189.97446,20977.026,2011
8,186.99179,23089.841,2011
9,177.70990,21731.449,2011


In [14]:
year_2012 <- subset(bikedata, yr == 1)
cnt_summary_2012 <- aggregate(cnt ~ mnth, data = year_2012, FUN = function(x) c(mean = mean(x), var = var(x)))
cnt_summary_2012$mean <- cnt_summary_2012$cnt[,1]
cnt_summary_2012$var <- cnt_summary_2012$cnt[,2]
cnt_summary_2012$yr <- 2012
cnt_summary_2012$yr <- factor(cnt_summary_2012$yr) 
cnt_summary_2012 = cnt_summary_2012[c("mnth", "mean", "var", "yr")]
cnt_summary_2012                   

mnth,mean,var,yr
<int>,<dbl>,<dbl>,<fct>
1,130.5587,14351.25,2012
2,149.0419,18032.86,2012
3,221.9044,38013.55,2012
4,242.6518,44495.40,2012
5,263.2594,45834.14,2012
6,281.7083,49466.60,2012
7,273.6653,45863.84,2012
8,288.3105,51927.01,2012
9,303.5736,62430.32,2012


In [15]:
summary = rbind(cnt_summary_2011, cnt_summary_2012)
head(summary)

,mnth,mean,var,yr
,<int>,<dbl>,<dbl>,<fct>
1,1,55.50727,2363.968,2011
2,2,74.29122,4048.268,2011
3,3,87.73288,6221.974,2011
4,4,131.94715,15056.421,2011
5,5,182.55511,21431.031,2011
6,6,199.32222,24050.936,2011


In [ ]:
# draw the plot with reduced x-axis ticks
ggplot(summary, aes(x = yr, y = var)) + 
  geom_line(size = 2, color = "red") +
  labs(x = "Year", y = "Variance of cnt") +
  theme(axis.title.x = element_text(size = 11), 
        axis.title.y = element_text(size = 11), 
        plot.title = element_text(size = 15, hjust = 0.5),
        axis.text.y = element_text(size = 7))

Variance is different during the duration of the data. We can see it increased in 2012 considerable as compared to 2011.

In [ ]:
mydata = bikedata %>% 
         select(cnt, season, mnth, hr, holiday, weekday, weathersit, temp)
head(mydata)

# dteday
# yr

## Test-Train split

In [ ]:
set.seed(2023)
sample.index <- sample(nrow(mydata),nrow(mydata)*0.80, replace = FALSE)
bike.train <- mydata[sample.index,]
bike.test <- mydata[-sample.index,]

In [ ]:
head(bike.test)

In [ ]:
head(bike.train)

## LM : Before converting variables to factors

In [ ]:
modlm1 = lm(cnt~., data=bike.train)
summary(modlm1)
(yhat_modlm1 <- round(predict(modlm1, bike.test[,-1], type="response")))
cat("Min predicted value:", min(yhat_modlm1))
mse1 = mean((yhat_modlm1-bike.test[,1])^2) # calculating mse value
cat("\nMSE:", mse1)

The minimum predicted value is negative which seems appropriate for 'cnt' variable.

Converting the variables to factors:

In [ ]:
mydata$mnth = as.factor(mydata$mnth)
mydata$hr = as.factor(mydata$hr)
mydata$weathersit = as.factor(mydata$weathersit)
mydata$season = as.factor(mydata$season)
# mydata$weekday = factor(mydata$weekday)

In [ ]:
head(mydata)

##  LM : After converting variables to factors (lower MSE and BIC score)

In [ ]:
modlm2 = lm(cnt~., data=bike.train)
summary(modlm2)
(yhat_modlm2 <- round(predict(modlm2, bike.test[,-1], type="response")))
cat("Min predicted value:", min(yhat_modlm2))
mse2 = round(mean((yhat_modlm2-bike.test[,1])^2),2)
cat("\nMSE:", mse2)

The minimum predicted value is negative which seems inappropriate for 'cnt' variable as the number of bike users cannot be negative.

## Best subset selection for model with factors

In [ ]:
aic_back <- step(modlm2, direction="both", trace= FALSE) # by default direction is "backward", trace = TRUE
summary(aic_back)
plot(aic_back)

In [ ]:
library(lmtest)
reset(modlm2)

## GLM - Poisson (factors)

In [ ]:
modglm1 = glm(cnt~., data=bike.train, family=poisson)
summary(modglm1)
yhat_modglm1 <- round(predict(modglm1, bike.test[,-1], type="response"))
cat("Min predicted value:", min(yhat_modglm1))
mse3 = round(mean((yhat_modglm1-bike.test[,1])^2),2)
cat("\nMSE:", mse3)
plot(modglm1)

The minimum predicted value is positive which seems appropriate for 'cnt' variable.

In [ ]:
library(lmtest)
reset(modglm1)

In [ ]:
df <- data.frame(Model = c("LM (no factors)", "LM (with factors)", "GLM (Poisson)"),
                 BIC = c(BIC(modlm1), BIC(modlm2), BIC(modglm1)),
                 MSE = c(get_mse(modlm1), get_mse(modlm2), get_mse(modglm1)))
df

In [ ]:
summary

In [6]:
options(repr.plot.width = 12, repr.plot.height = 8)

ggplot(summary, aes(x = mnth, y = var)) + 
  geom_line(size = 2, color = "red") + facet_wrap(~yr)
  labs(x = "Month", y = "Variance of cnt") +
  theme(axis.title.x = element_text(size = 11), 
        axis.title.y = element_text(size = 11), 
        plot.title = element_text(size = 15, hjust = 0.5),
        axis.text.y = element_text(size = 7)) 
# scale_x_discrete(labels = function(x) format(as.Date(paste0(x, "-01")), "%b"))

ERROR: [1m[33mError[39m in `ggplot()`:[22m
[1m[22m[33m![39m `data` cannot be a function.
[36mℹ[39m Have you misspelled the `data` argument in `ggplot()`


## 